In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0   479k      0  0:02:51  0:02:51 --:--:-- 2024k22.0M    0     0   368k      0  0:03:43  0:01:01  0:02:42  409k.2M   76 61.1M    0     0   411k      0  0:03:19  0:02:32  0:00:47  610k


In [2]:
!rm -r aclImdb/train/unsup

In [3]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [5]:
import os, pathlib, shutil, random


base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir/category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for file_name in val_files:
        shutil.move(train_dir / category / file_name, val_dir / category / file_name) 

In [1]:
import keras
from keras.utils import text_dataset_from_directory
batch_size = 32

train_ds = text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.


2022-09-08 13:04:19.055470: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-09-08 13:04:19.055508: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: nitec-ABRA-A5-V15-2
2022-09-08 13:04:19.055516: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: nitec-ABRA-A5-V15-2
2022-09-08 13:04:19.055609: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.65.1
2022-09-08 13:04:19.055630: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.65.1
2022-09-08 13:04:19.055635: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 515.65.1
2022-09-08 13:04:19.055974: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in p

Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [3]:
for inputs,targets in train_ds:
    print("inputs.shape", inputs.shape)
    print("inputs.dtype", inputs.dtype)
    print("targets.shape", targets.shape)
    print("targets.dtype", targets.dtype)
    print("inputs[0]", inputs[0])
    print("targets[0]", targets[0])
    break

inputs.shape (32,)
inputs.dtype <dtype: 'string'>
targets.shape (32,)
targets.dtype <dtype: 'int32'>
inputs[0] tf.Tensor(b'My abiding love of Italian actress Lucianna Paluzzi, who helped jump-start my puberty with her performance in 1965\'s "Thunderball," has led me to some fairly unusual places. Case in point, this British curiosity from 1959, "Carlton-Browne of the F.O.," which features Lucianna in one of her earlier roles. She plays a princess in this one, although the picture is actually a showcase for the talents of Terry-Thomas and Peter Sellers, both of whose stars were certainly on the rise at this point. In this cute, often very funny film, we learn of the Madeira-like island nation of Gaillardia, which had been a British colony until 1916 and then universally forgotten. Forty-three years later, however, it becomes the center of worldwide attention and international espionage when valuable cobalt deposits are discovered there, and Her Majesty sends the bumbling Carlton-Browne 

In [4]:
from keras.layers import TextVectorization
text_vectorization = TextVectorization(
    max_tokens= 20000, output_mode='multi_hot'
)

In [5]:
text_only_train_ds = train_ds.map(lambda x, y:x)
text_vectorization.adapt(text_only_train_ds)

binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

binary_1gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [6]:
import keras
from keras import layers

def get_model(max_tokens:int = 20000, hidden_dim: int = 16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation='relu')(inputs)
    x =  layers.Dropout(.5)(x)
    
    outputs = layers.Dense(1, activation='sigmoid')(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer = "rmsprop", loss = "binary_crossentropy", metrics = "accuracy")
    return model

In [7]:
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


In [8]:
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.keras", save_best_only=True)
]

model.fit(binary_1gram_train_ds.cache(), validation_data=binary_1gram_val_ds.cache(), epochs = 10, callbacks = callbacks)

model = keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 26s 41ms/step - loss: 0.4177 - accuracy: 0.8220 - val_loss: 0.2872 - val_accuracy: 0.8870
Epoch 2/10
625/625 [==============================] - 189s 298ms/step - loss: 0.2912 - accuracy: 0.8963 - val_loss: 0.2815 - val_accuracy: 0.8932
Epoch 3/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2630 - accuracy: 0.9110 - val_loss: 0.2852 - val_accuracy: 0.8908
Epoch 4/10
625/625 [==============================] - 2s 3ms/step - loss: 0.2440 - accuracy: 0.9180 - val_loss: 0.3023 - val_accuracy: 0.8880
Epoch 5/10
625/625 [==============================] - 2s 3ms/step - loss: 0.2395 - accuracy: 0.9219 - val_loss: 0.3110 - val_accuracy: 0.8906
Epoch 6/10
625/625 [==============================] - 2s 3ms/step - loss: 0.2342 - accuracy: 0.9222 - val_loss: 0.3216 - val_accuracy: 0.8874
Epoch 7/10
625/625 [==============================] - 2s 3ms/step - loss: 0.2261 - accuracy: 0.9281 - val_loss: 0.3269 - val_accuracy: 0.8896


In [10]:
# TextVectorization can also be applied to N-grams

text_vectorization = TextVectorization(
    ngrams= 2,
    max_tokens=20000,
    output_mode='multi_hot',
)

In [11]:
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(lambda x,y: (text_vectorization(x), y), num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(lambda x,y: (text_vectorization(x), y), num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(lambda x,y: (text_vectorization(x), y), num_parallel_calls=4)

In [12]:
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_2gram.keras", save_best_only=True)
]

model.fit(binary_2gram_train_ds.cache(), validation_data=binary_2gram_val_ds.cache(), epochs = 10, callbacks = callbacks)

model = keras.models.load_model("binary_2gram.keras")
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_2 (Dense)             (None, 16)                320016    
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 173s 277ms/step - loss: 0.3759 - accuracy: 0.8443 - val_loss: 0.2677 - val_accuracy: 0.8918
Epoch 2/10
625/625 [===================

: 